# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rajapur,25.3833,81.1500,89.49,59,73,6.96,IN,1690164252
1,1,port-aux-francais,-49.3500,70.2167,42.57,94,100,38.68,TF,1690164253
2,2,whitehorse,60.7161,-135.0538,57.97,94,100,8.05,CA,1690164079
3,3,blackmans bay,-43.0167,147.3167,55.72,67,100,7.09,AU,1690164253
4,4,klyuchi,52.2667,79.1667,75.63,37,46,4.76,RU,1690164253


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 70) & (city_data_df["Cloudiness"] < 25) & (city_data_df["Wind Speed"] < 15)]

# Drop any rows with null values
ideal_weather_cities_complete = ideal_weather_cities.dropna()

# Display sample data
ideal_weather_cities_complete

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,saveh,35.0213,50.3566,82.26,19,5,4.05,IR,1690164254
23,23,avarua,-21.2078,-159.7750,78.85,83,1,14.97,CK,1690164261
40,40,acarau,-2.8856,-40.1200,77.92,81,9,12.24,BR,1690164266
43,43,college,64.8569,-147.8028,87.58,45,0,1.01,US,1690164005
49,49,ribeira grande,38.5167,-28.7000,71.78,84,10,5.99,PT,1690164268
...,...,...,...,...,...,...,...,...,...,...
530,530,hilvan,37.5869,38.9551,83.28,29,0,11.50,TR,1690164377
531,531,vermillion,42.7794,-96.9292,78.96,68,2,7.92,US,1690164377
532,532,chlef,36.1652,1.3345,88.12,25,0,0.00,DZ,1690164378
545,545,ta`u,-14.2336,-169.5144,79.66,72,6,13.40,AS,1690164381


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_complete[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\vasqu\AppData\Local\Temp\ipykernel_31548\182853606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
6,saveh,IR,35.0213,50.3566,19,
23,avarua,CK,-21.2078,-159.7750,83,
40,acarau,BR,-2.8856,-40.1200,81,
43,college,US,64.8569,-147.8028,45,
49,ribeira grande,PT,38.5167,-28.7000,84,
...,...,...,...,...,...,...
530,hilvan,TR,37.5869,38.9551,29,
531,vermillion,US,42.7794,-96.9292,68,
532,chlef,DZ,36.1652,1.3345,25,
545,ta`u,AS,-14.2336,-169.5144,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saveh - nearest hotel: هتل صدرا
avarua - nearest hotel: Paradise Inn
acarau - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
ribeira grande - nearest hotel: Hospedaria JSF
mayya - nearest hotel: Гостиница
waddan - nearest hotel: No hotel found
fort mcmurray - nearest hotel: Nomad Hotel and Suites
georgetown - nearest hotel: Page 63 hostel
howard springs - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
badger - nearest hotel: Hotel North Pole
hamilton - nearest hotel: North Vista Manor
verkhnevilyuysk - nearest hotel: No hotel found
nadym - nearest hotel: Янтарная
north bend - nearest hotel: The Mill Casino Hotel
okha - nearest hotel: Сфера
taulaga - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
hua hin - nearest hotel: Anchan
atafu village - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
eregli - nearest hotel: No hotel found
posto

,City,Country,Lat,Lng,Humidity,Hotel Name
6,saveh,IR,35.0213,50.3566,19,هتل صدرا
23,avarua,CK,-21.2078,-159.7750,83,Paradise Inn
40,acarau,BR,-2.8856,-40.1200,81,No hotel found
43,college,US,64.8569,-147.8028,45,BEST WESTERN PLUS Pioneer Park Inn
49,ribeira grande,PT,38.5167,-28.7000,84,Hospedaria JSF
...,...,...,...,...,...,...
530,hilvan,TR,37.5869,38.9551,29,No hotel found
531,vermillion,US,42.7794,-96.9292,68,Best Western Vermillion Inn
532,chlef,DZ,36.1652,1.3345,25,فندق الونشريس
545,ta`u,AS,-14.2336,-169.5144,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols = ["City", "Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)